**OpenStakc Web APIを使って仮想マシンを操作してみよう** 　　

API referenceは　　

https://developer.openstack.org/ja/api-guide/quick-start/　　

https://developer.openstack.org/api-ref/identity/v2/　　

https://developer.openstack.org/api-ref/compute/

**まず　APIを使う時の認証に必要なtokenを　usernameとpasswordを　identity manager に問い合わせて獲得** 　　

token のところの　idに現れる　文字列がそれ。但し、一定時間経つと　expireするので使えなくなる　　

token は以降　APIを呼び出すたびに必要となる

In [ ]:
!sudo apt-get update;sudo apt-get install curl -y

In [1]:
%env OS_AUTH_URL=http://133.8.140.97:5000/v3
%env OS_PROJECT_NAME=students
%env OS_USERNAME=jxta
%env OS_PASSWORD=Gunma-u#
%env OS_PROJECT_DOMAIN_NAME=default
%env OS_USER_DOMAIN_NAME=default
%env NOVA_END_POINT=http://133.8.140.97:8774/v2.1/8126d5a8dd0743f9afc9a9c5975622d8

env: OS_AUTH_URL=http://133.8.140.97:5000/v3
env: OS_PROJECT_NAME=students
env: OS_USERNAME=jxta
env: OS_PASSWORD=Gunma-u#
env: OS_PROJECT_DOMAIN_NAME=default
env: OS_USER_DOMAIN_NAME=default
env: NOVA_END_POINT=http://133.8.140.97:8774/v2.1/8126d5a8dd0743f9afc9a9c5975622d8


In [2]:
!curl -v -s -X POST $OS_AUTH_URL/auth/tokens?nocatalog   -H "Content-Type: application/json"   -d '{ "auth": { "identity": { "methods": ["password"],"password": {"user": {"domain": {"name": "'"$OS_USER_DOMAIN_NAME"'"},"name": "'"$OS_USERNAME"'", "password": "'"$OS_PASSWORD"'"} } }, "scope": { "project": { "domain": { "name": "'"$OS_PROJECT_DOMAIN_NAME"'" }, "name":  "'"$OS_PROJECT_NAME"'" } } }}' \
| python -m json.tool

*   Trying 133.8.140.97:5000...
* TCP_NODELAY set
* Connected to 133.8.140.97 (133.8.140.97) port 5000 (#0)
> POST /v3/auth/tokens?nocatalog HTTP/1.1
> Host: 133.8.140.97:5000
> User-Agent: curl/7.68.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 230
> 
} [230 bytes data]
* upload completely sent off: 230 out of 230 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 201 CREATED
< Date: Wed, 12 Aug 2020 05:08:06 GMT
< Server: Apache
< X-Subject-Token: gAAAAABfM3k2SHruCU3fddsMOOSXomoEuae0tELVV4EbAEUhNwP2WevvS6F5X52OpoCNDJl9usKfLE5jwpDhCAwuBot6pcjh3ehFQeN7MKm6J2ZobItBzNS8rkDDt9VSFzHxGswoo6QeOKYUdwb2OP2EFHOdmHCS3v4ih707LlcoKl6Db-7KRtg
< Vary: X-Auth-Token
< x-openstack-request-id: req-e301805f-dd6a-4c44-9f97-3263bd226aaf
< Content-Length: 529
< Content-Type: application/json
< 
{ [529 bytes data]
* Connection #0 to host 133.8.140.97 left intact
{
    "token": {
        "is_domain": false,
        "methods": [
            "password"
        ],
        "roles": [
 

In [3]:
%env X_Auth_Token="gAAAAABfM3k2SHruCU3fddsMOOSXomoEuae0tELVV4EbAEUhNwP2WevvS6F5X52OpoCNDJl9usKfLE5jwpDhCAwuBot6pcjh3ehFQeN7MKm6J2ZobItBzNS8rkDDt9VSFzHxGswoo6QeOKYUdwb2OP2EFHOdmHCS3v4ih707LlcoKl6Db-7KRtg"

env: X_Auth_Token="gAAAAABfM3k2SHruCU3fddsMOOSXomoEuae0tELVV4EbAEUhNwP2WevvS6F5X52OpoCNDJl9usKfLE5jwpDhCAwuBot6pcjh3ehFQeN7MKm6J2ZobItBzNS8rkDDt9VSFzHxGswoo6QeOKYUdwb2OP2EFHOdmHCS3v4ih707LlcoKl6Db-7KRtg"


In [4]:
!curl -s -H  "X-Auth-Token:$X_Auth_Token" $NOVA_END_POINT/flavors | python -m json.tool

{
    "flavors": [
        {
            "id": "1",
            "links": [
                {
                    "href": "http://133.8.140.97:8774/v2.1/8126d5a8dd0743f9afc9a9c5975622d8/flavors/1",
                    "rel": "self"
                },
                {
                    "href": "http://133.8.140.97:8774/8126d5a8dd0743f9afc9a9c5975622d8/flavors/1",
                    "rel": "bookmark"
                }
            ],
            "name": "m1.tiny"
        },
        {
            "id": "2",
            "links": [
                {
                    "href": "http://133.8.140.97:8774/v2.1/8126d5a8dd0743f9afc9a9c5975622d8/flavors/2",
                    "rel": "self"
                },
                {
                    "href": "http://133.8.140.97:8774/8126d5a8dd0743f9afc9a9c5975622d8/flavors/2",
                    "rel": "bookmark"
                }
            ],
            "name": "m1.small"
        },
        {
            "id": "3",
            "links": [
    

**server 一覧　を取得し、自分の使っている仮想マシンの　server_idを得る**

In [5]:
!curl -s -H  "X-Auth-Token:$X_Auth_Token" $NOVA_END_POINT/servers | python -m json.tool

{
    "servers": [
        {
            "id": "258fdd07-c641-421e-9667-8e38be9621a8",
            "links": [
                {
                    "href": "http://133.8.140.97:8774/v2.1/8126d5a8dd0743f9afc9a9c5975622d8/servers/258fdd07-c641-421e-9667-8e38be9621a8",
                    "rel": "self"
                },
                {
                    "href": "http://133.8.140.97:8774/8126d5a8dd0743f9afc9a9c5975622d8/servers/258fdd07-c641-421e-9667-8e38be9621a8",
                    "rel": "bookmark"
                }
            ],
            "name": "yoko-777"
        }
    ]
}


**取得したserver_idを指定して、その仮想マシンの詳細情報を取得**

In [6]:
%env SERVER_ID=258fdd07-c641-421e-9667-8e38be9621a8
!curl -s -H  "X-Auth-Token:$X_Auth_Token" $NOVA_END_POINT/servers/$SERVER_ID | python -m json.tool

env: SERVER_ID=258fdd07-c641-421e-9667-8e38be9621a8
{
    "server": {
        "OS-EXT-STS:task_state": null,
        "addresses": {
            "private_network_with_DNS": [
                {
                    "OS-EXT-IPS-MAC:mac_addr": "fa:16:3e:26:98:2b",
                    "version": 4,
                    "addr": "192.168.102.134",
                    "OS-EXT-IPS:type": "fixed"
                },
                {
                    "OS-EXT-IPS-MAC:mac_addr": "fa:16:3e:26:98:2b",
                    "version": 4,
                    "addr": "133.8.140.235",
                    "OS-EXT-IPS:type": "floating"
                }
            ]
        },
        "links": [
            {
                "href": "http://133.8.140.97:8774/v2.1/8126d5a8dd0743f9afc9a9c5975622d8/servers/258fdd07-c641-421e-9667-8e38be9621a8",
                "rel": "self"
            },
            {
                "href": "http://133.8.140.97:8774/8126d5a8dd0743f9afc9a9c5975622d8/servers/258fdd07-c641-421

**その仮想マシンの名前を変更してみましょう** 

**そして実際に変化していることを OpenStackのDash Boardで確認してみよう**  

OpenStackのDash Board : http://133.8.140.97/dashboard

In [ ]:
!curl -X PUT -H "Content-Type: application/json" -H "X-Auth-Token:$X_Auth_Token"  -d '{"server": { "name" : "yoko-7777" }}' $NOVA_END_POINT/servers/$SERVER_ID|python -m json.tool

**仮想マシンを停止してみよう**

**そして実際に停止していることを OpenStackのDash Boardで確認してみよう**  

**また、sshできなくなっていることを確認してみよう**

OpenStackのDash Board : http://133.8.140.97/dashboard


In [ ]:
!curl -X POST -H "Content-Type: application/json" -H "X-Auth-Token:$X_Auth_Token"  -d '{"os-stop": null}' $NOVA_END_POINT/servers/$SERVER_ID/action

**仮想マシンを再稼働してみよう**

**そして実際に再稼働していることを OpenStackのDash Boardで確認してみよう**  

**sshが再度できるようになっていることを確認してみよう**

OpenStackのDash Board : http://133.8.140.97/dashboard


In [ ]:
!curl -X POST -H "Content-Type: application/json" -H "X-Auth-Token:$X_Auth_Token"  -d '{"os-start": null}' $NOVA_END_POINT/servers/$SERVER_ID/action